# Querying Azure Sentinel via REST API using a Personal Access Token (PAT)

<table align="left" style="margin-left:25px">
  <tr>
    <th>Type</th>
    <th>Title</th>
    <th>Description</th>
  </tr>
  <tr>
      <td>Documentation</td>
      <td><b><a href="https://techcommunity.microsoft.com/t5/azure-sentinel/extending-azure-sentinel-apis-integration-and-management/ba-p/1116885">Extending Azure Sentinel: APIs, Integration and management automation</a></td>
    <td>Its highly recommended to follow this guide to get familiar with navigating and using Jupyter Notebooks. </td>
  </tr>
  <tr>
      <td>Tech Community</td>
    <td><b><a href="../../PowerShell/Getting-Started-README.ipynb">Access Azure Sentinel Log Analytics via API (Part 1)</a></td>
    <td>@rinure wrote a good article explaining how to query the API. Use this as a reference to understand how to query the Azure Sentinel API. </td>
  </tr>
</table> 

## Set your variables here

In [9]:
############ This is the credentials used to authenticate with the Graph API ###################
#$credentials = @{
#    tenantDomain = "domain.com"
#    tenantId = "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxx"        ## AAD Tenant ID
#    client_id = "ae4fa870-23af-4ae5-a6ed-xxxxx"     ## Application Client ID
#    client_secret = "x~xxxxxxx~xxxxx~xxxxxxxxx-xxx"   ## Application Client Secret
#}

$sentinelWorkspaceId = "a8960181-0bde-4955-9f94-2c12a36cc900"

## Import required modules

In [2]:
$modules = @('pp-core','graph') ## These are the modules to import 
$credentialPath = "~/.credentials/graph.credential" ## Where to store credentials
################## Import Graph Authentication Module #############################
ForEach ($module in $modules) {
    Import-Module "~/Notebooks/Powershell-Playground/PowerShell/custom-modules/$module" -Force  ## import module 
    Get-Command -Module "$module"                                                        ## list commands in this module 
}

$credentials = New-StoreCredentials -credentialPath $credentialPath -credentialArray $credentials


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

 ·········


In [4]:
## Get a token if authenticates properly.
$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://api.loganalytics.io"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a appliation ID and secret to authenticate
}

$tokenResponse = Invoke-GraphAuthentication -authParams $authParams


Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/de40cf7e-ad5f-4245-a317-14be39cbb0ef/oauth2/token


Received Token!
Connected and Access Token received and will expire 


### Put your Azure Sentinel Hunting Query here

In [6]:
## Enter your query here
$query = @"

SecurityAlert

"@

$timeSpan = "" ## You can leave blank or specify a date. But something HAS to be there. 
 
## This is the query converted to JSON format
$body = (@{
   query = $Query
   timespan = $timeSpan  
} | ConvertTO-JSON)
Write-Host -ForeGround Yellow "`nQuery"; $Query
Write-Host -ForeGround Yellow "JSON"; $body


Query

SecurityAlert

JSON
{
  "query": "\nSecurityAlert\n",
  "timespan": ""
}


## Query Graph API and insert into $graphresponse variable

In [12]:
$URI = "https://api.loganalytics.io/v1/workspaces/$sentinelWorkspaceId/query"
$header = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
}

$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $header -body $body -ErrorAction Stop)
$Result.tables.rows | Select -First 1

a8960181-0bde-4955-9f94-2c12a36cc900

Wednesday, June 10, 2020 7:43:02 PM
DLP - Sharepoint - Financial sensitive file(s) shared externally
DLP - Sharepoint - Financial sensitive file(s) shared externally
High
File policy 'DLP - Sharepoint - Financial sensitive file(s) shared externally' was matched by 'Credit_Card_Form_SP_20200610-T0239.docx'
MCAS
Microsoft
B782FAF8-F546-37A1-979B-340DE4AFF1C2
56fb8160-5d9c-c29b-be35-be169678b9bf


MCAS_ALERT_CABINET_EVENT_MATCH_FILE
Unknown

False
Wednesday, June 10, 2020 7:42:56 PM
Wednesday, June 10, 2020 7:42:56 PM
Wednesday, June 10, 2020 7:43:02 PM

{
  "Cloud Applications": "Microsoft SharePoint Online"
}
[
  {
    "$id": "3",
    "AppId": 20892,
    "Name": "Microsoft SharePoint Online",
    "InstanceName": "Microsoft SharePoint Online",
    "Type": "cloud-application"
  },
  {
    "$id": "4",
    "Name": "bob",
    "UPNSuffix": "jingtoso.com",
    "AadUserId": "6836ed6d-2a15-4889-ad24-fbda46b5ffda",
    "Type": "account"
  }
]
Detection


[
  